In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
configuration_path = "data/configuration.json"
trips_path = "data/trips.csv"
output_path = "data/output/"
output_file = output_path + "copert.xlsx"
year = 2022

## Read data

In [ ]:
with open(configuration_path) as f:
    configuration = json.load(f)

In [ ]:
df_trips = pd.read_csv(trips_path, sep = ";")

In [ ]:
f_midnight = df_trips["departure_time"] > 24 * 3600
while np.count_nonzero(f_midnight) > 0:
    df_trips.loc[f_midnight, "departure_time"] -= 24 * 3600
    f_midnight = df_trips["departure_time"] > 24 * 3600

## Prepare general data

In [ ]:
months = [
    "January", "February", "March", "April", "May", "June", "July", 
    "August", "September", "October", "November", "December"]

In [ ]:
df_min_temperature = pd.DataFrame.from_records([
    { "Month": month, str(year): temperature }
    for temperature, month in zip(configuration["temperature"]["min"], months)
])

df_max_temperature = pd.DataFrame.from_records([
    { "Month": month, str(year): temperature }
    for temperature, month in zip(configuration["temperature"]["max"], months)
])

In [ ]:
df_humidity = pd.DataFrame.from_records([
    { "Month": month, str(year): humidity }
    for humidity, month in zip(configuration["humidity"], months)
])

In [ ]:
df_sheets = pd.DataFrame.from_records([
    { "SHEET_NAME": "STOCK", "Unit": "[n]" },
    { "SHEET_NAME": "MEAN_ACTIVITY", "Unit": "[km]" },
    { "SHEET_NAME": "URBAN_OFF_PEAK_SPEED", "Unit": "[km/h]" },
    { "SHEET_NAME": "URBAN_PEAK_SPEED", "Unit": "[km/h]" },
    { "SHEET_NAME": "URBAN_OFF_PEAK_SHARE", "Unit": "[%]" },
    { "SHEET_NAME": "URBAN_PEAK_SHARE", "Unit": "[%]" },
    { "SHEET_NAME": "MIN_TEMPERATURE", "Unit": "[°C]" },
    { "SHEET_NAME": "MAX_TEMPERATURE", "Unit": "[°C]" },
    { "SHEET_NAME": "HUMIDITY", "Unit": "[%]" },
    { "SHEET_NAME": "FUEL_TANK_SIZE", "Unit": "[l]" },
])

## By mode

In [ ]:
mode_offpeak_speeds = { mode: np.nan for mode in configuration["mode_mapping"].keys() }
mode_peak_speeds = { mode: np.nan for mode in configuration["mode_mapping"].keys() }

mode_offpeak_share = { mode: np.nan for mode in configuration["mode_mapping"].keys() }
mode_peak_share = { mode: np.nan for mode in configuration["mode_mapping"].keys() }

mode_counts = { mode: 0 for mode in configuration["mode_mapping"].keys() }
mode_distances = { mode: 0.0 for mode in configuration["mode_mapping"].keys() }

f_peak = np.zeros((len(df_trips),), dtype = bool)

for peak in configuration["peaks"]:
    f_peak |= df_trips["departure_time"].between(peak["start"], peak["end"])

for mode, config in configuration["mode_mapping"].items():
    f = df_trips["mode"] == mode
    sampling_rate = config["sampling_rate"]
    mode_counts[mode] = len(df_trips[f]["person_id"].unique()) / sampling_rate
    mode_distances[mode] += df_trips[f]["routed_distance"].sum() * 1e-3 / sampling_rate
    
    peak_speeds = (df_trips[f & f_peak]["routed_distance"] * 1e-3) / (df_trips[f & f_peak]["travel_time"] / 3600)
    offpeak_speeds = (df_trips[f & ~f_peak]["routed_distance"] * 1e-3) / (df_trips[f & ~f_peak]["travel_time"] / 3600)

    if not peak_speeds.empty or offpeak_speeds.empty: continue

    peak_speeds = peak_speeds.values
    offpeak_speeds = offpeak_speeds.values
    
    peak_speeds = peak_speeds[np.isfinite(peak_speeds)]
    offpeak_speeds = offpeak_speeds[np.isfinite(offpeak_speeds)]
    
    peak_speeds = np.sort(peak_speeds)
    offpeak_speeds = np.sort(offpeak_speeds)
    
    peak_speeds = peak_speeds[peak_speeds >= np.quantile(peak_speeds, 0.1)]
    offpeak_speeds = offpeak_speeds[offpeak_speeds >= np.quantile(offpeak_speeds, 0.1)]
    
    mode_offpeak_speeds[mode] = np.mean(peak_speeds)
    mode_peak_speeds[mode] = np.mean(offpeak_speeds)
    
    mode_offpeak_share[mode] = np.count_nonzero(f & ~f_peak) / np.count_nonzero(f)
    mode_peak_share[mode] = np.count_nonzero(f & f_peak) / np.count_nonzero(f)

In [ ]:
df_stock = []
df_mean_activity = []

df_peak_speed = []
df_offpeak_speed = []

df_peak_share = []
df_offpeak_share = []

df_fuel_tank_size = []

In [ ]:
for mode, config in configuration["mode_mapping"].items():
    common = {
        "Category": config["category"],
        "Fuel": config["fuel"],
        "Segment": config["segment"],
        "Euro Standard": config["euro_standard"]
    }
    
    peak_share = {}
    peak_share.update(common)
    peak_share[str(year)] = mode_peak_share[mode] * 100.0
    df_peak_share.append(peak_share)
    
    offpeak_share = {}
    offpeak_share.update(common)
    offpeak_share[str(year)] = mode_offpeak_share[mode] * 100.0
    df_offpeak_share.append(offpeak_share)
    
    peak_speed = {}
    peak_speed.update(common)
    peak_speed[str(year)] = mode_peak_speeds[mode]
    df_peak_speed.append(peak_speed)
    
    offpeak_speed = {}
    offpeak_speed.update(common)
    offpeak_speed[str(year)] = mode_offpeak_speeds[mode]
    df_offpeak_speed.append(offpeak_speed)
    
    fuel_tank_size = {}
    fuel_tank_size.update(common)
    fuel_tank_size[str(year)] = config["fuel_tank_size"]
    df_fuel_tank_size.append(fuel_tank_size)
    
    stock = {}
    stock.update(common)
    stock[str(year)] = mode_counts[mode]
    df_stock.append(stock)
    
    mean_activity = {}
    mean_activity.update(common)
    mean_activity[str(year)] = mode_distances[mode]
    df_mean_activity.append(mean_activity)

In [ ]:
df_peak_share = pd.DataFrame.from_records(df_peak_share)
df_offpeak_share = pd.DataFrame.from_records(df_offpeak_share)
df_peak_speed= pd.DataFrame.from_records(df_peak_speed)
df_offpeak_speed = pd.DataFrame.from_records(df_offpeak_speed)
df_fuel_tank_size = pd.DataFrame.from_records(df_fuel_tank_size)
df_stock = pd.DataFrame.from_records(df_stock)
df_mean_activity = pd.DataFrame.from_records(df_mean_activity)

## Output

In [ ]:
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    df_sheets.to_excel(writer, index = False, sheet_name = "SHEETS")
    df_stock.to_excel(writer, index = False, sheet_name = "STOCK")
    df_mean_activity.to_excel(writer, index = False, sheet_name = "MEAN_ACTIVITY")
    df_offpeak_speed.to_excel(writer, index = False, sheet_name = "URBAN_OFFPEAK_SPEED")
    df_peak_speed.to_excel(writer, index = False, sheet_name = "URBAN_PEAK_SPEED")
    df_offpeak_share.to_excel(writer, index = False, sheet_name = "URBAN_OFFPEAK_SHARE")
    df_peak_share.to_excel(writer, index = False, sheet_name = "URBAN_PEAK_SHARE")
    df_min_temperature.to_excel(writer, index = False, sheet_name = "MIN_TEMPERATURE")
    df_max_temperature.to_excel(writer, index = False, sheet_name = "MAX_TEMPERATURE")
    df_humidity.to_excel(writer, index = False, sheet_name = "HUMIDITY")
    df_fuel_tank_size.to_excel(writer, index = False, sheet_name = "FUEL_TANK_SIZE")